<a href="https://colab.research.google.com/github/abinmaria/Quantum-and-Quantum_ML/blob/main/Embedding(Pennylane).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
!pip install pennylane



In [ ]:
import pennylane as qml
from pennylane import numpy as np


[# Basis embedding transform:1,1 to |11>

In [ ]:
# basis encoding
wires = range(2) # set num_qubits
basis_dev = qml.device("default.qubit",wires)

@qml.qnode(basis_dev)
def basis_encoder(data):
  qml.BasisEmbedding(data,wires)
  return qml.state()


In [ ]:
basis_encoder(3)


tensor([0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j], requires_grad=True)

# Amplitude encoding

In [ ]:
# amplitude encoding
wires = range(2) # set num_qubits
amp_dev = qml.device("default.qubit",wires)

@qml.qnode(amp_dev)
def amp_encoder(data):
  qml.AmplitudeEmbedding(data,wires)
  return qml.state()

In [ ]:
amp_encoder([1/np.sqrt(2),1/np.sqrt(2),0,0])


tensor([0.70710678+0.j, 0.70710678+0.j, 0.        +0.j, 0.        +0.j], requires_grad=True)

In [ ]:
#When size(data)!=2^n
wires = range(2) # set num_qubits
amp_dev = qml.device("default.qubit",wires)

@qml.qnode(amp_dev)
def amp_encoder1(data):
  qml.AmplitudeEmbedding(data,wires,pad_with=0)
  return qml.state()

In [ ]:
amp_encoder1([1/np.sqrt(2),0,1/np.sqrt(2)])


tensor([0.70710678+0.j, 0.        +0.j, 0.70710678+0.j, 0.        +0.j], requires_grad=True)

In [ ]:
#When Born's rule is not satisfied
wires = range(2) # set num_qubits
amp_dev = qml.device("default.qubit",wires)

@qml.qnode(amp_dev)
def amp_encoder2(data):
  qml.AmplitudeEmbedding(data,wires,normalize=True)
  return qml.state()

In [ ]:
amp_encoder2([1.5+1j,2.5,1,1])


tensor([0.44232587+0.29488391j, 0.73720978+0.j        ,
        0.29488391+0.j        , 0.29488391+0.j        ], requires_grad=True)

# Angle embedding

In [ ]:
wires = range(1) # set num_qubits
angle_dev = qml.device("default.qubit",wires)

@qml.qnode(angle_dev)
def angle_encoder(data):
  qml.AngleEmbedding(features=data,wires=wires,rotation='X')
  return qml.state()

In [ ]:
angle_encoder([1])


tensor([0.87758256+0.j        , 0.        -0.47942554j], requires_grad=True)

In [ ]:
print(qml.draw(qnode=angle_encoder,expansion_strategy="device")([1]))


0: ──RX(1.00)─┤  State


/usr/local/lib/python3.10/dist-packages/pennylane/drawer/draw.py:37: PennyLaneDeprecationWarning: The 'expansion_strategy' argument is deprecated and will be removed in version 0.39. Instead, use the 'level' argument which offers more flexibility and options.
  warnings.warn(


In [ ]:
# This is what actually happens
np.array([[np.cos(1/2), -1j*np.sin(1/2)],[-1j*np.sin(1/2),np.cos(1/2)]]).dot([1,0])


tensor([0.87758256+0.j        , 0.        -0.47942554j], requires_grad=True)

# Displacement embedding

In [ ]:
wires = range(2)
dis_dev = qml.device("default.gaussian",wires)

@qml.qnode(dis_dev)
def dis_encoder(data):
    qml.DisplacementEmbedding(features=data, wires=range(2),method='amplitude',c=0.1)
    qml.QuadraticPhase(0.1,wires=1)
    return qml.expval(qml.NumberOperator(wires=1))

In [ ]:
dis_encoder([2,4])


array(16.47877626)

In [ ]:
print(qml.draw(qnode=dis_encoder,expansion_strategy="device")([1,2+0.5j]))


0: ──D(1.00+0.00j,0.10+0.00j)──────────┤     
1: ──D(2.00+0.50j,0.10+0.00j)──P(0.10)─┤  <n>


/usr/local/lib/python3.10/dist-packages/pennylane/drawer/draw.py:37: PennyLaneDeprecationWarning: The 'expansion_strategy' argument is deprecated and will be removed in version 0.39. Instead, use the 'level' argument which offers more flexibility and options.
  warnings.warn(


# QAOA embedding

In [ ]:
qaoa_dev = qml.device("default.qubit",wires=range(2))

@qml.qnode(qaoa_dev)
def qaoa_encoder(data,weights):
  qml.QAOAEmbedding(features=data,weights=weights,wires=range(2),local_field="Y")
  return qml.state()

In [ ]:
# define features
features = [4+2j,5]
# get weights
shape = qml.QAOAEmbedding.shape(n_layers=2,n_wires=2)
print("This is the required shape: ",shape)
weights = np.random.random(shape)
print("Have a look at the weights: ")
print(weights)

This is the required shape:  (2, 3)
Have a look at the weights: 
[[0.96589016 0.08356106 0.50335919]
 [0.15347023 0.60538204 0.91405938]]


In [ ]:
qaoa_encoder(data=features,weights=weights)


tensor([ 3.30670611+2.31377007j, -1.93596589-7.35852993j,
         3.16080314+2.12353317j, -2.12678254-6.64637193j], requires_grad=True)

In [ ]:
print(qml.draw(qnode=qaoa_encoder,expansion_strategy="device")(features,weights))


0: ──RX(4.00+2.00j)─╭MultiRZ(0.97)──RY(0.08)──RX(4.00+2.00j)─╭MultiRZ(0.15)──RY(0.61)
1: ──RX(5.00+0.00j)─╰MultiRZ(0.97)──RY(0.50)──RX(5.00+0.00j)─╰MultiRZ(0.15)──RY(0.91)

───RX(4.00+2.00j)─┤ ╭State
───RX(5.00+0.00j)─┤ ╰State


/usr/local/lib/python3.10/dist-packages/pennylane/drawer/draw.py:37: PennyLaneDeprecationWarning: The 'expansion_strategy' argument is deprecated and will be removed in version 0.39. Instead, use the 'level' argument which offers more flexibility and options.
  warnings.warn(


# Train the QAOA embedding

In [ ]:
qaoa_dev = qml.device("default.qubit",wires=range(2))

@qml.qnode(qaoa_dev)
def trainable_qaoa_encoder(weights,data):
  qml.QAOAEmbedding(weights=weights,features=data,wires=range(2),local_field="Y")
  return qml.expval(qml.PauliZ(0))

In [ ]:
opt = qml.GradientDescentOptimizer()
steps = 10
for i in range(steps):
  weights = opt.step(lambda w: trainable_qaoa_encoder(data=features,weights=w),weights)
  print(f"This is step {i} and weights are {weights}")

This is step 0 and weights are [[1.01960663 0.0964962  0.50299308]
 [0.02913511 0.49863393 0.91405938]]
This is step 1 and weights are [[ 1.06066483  0.1039053   0.50292541]
 [-0.10432075  0.38627457  0.91405938]]
This is step 2 and weights are [[ 1.08534486  0.10511416  0.50318844]
 [-0.24266255  0.27056791  0.91405938]]
This is step 3 and weights are [[ 1.09126843  0.10017243  0.50391443]
 [-0.38106002  0.15405151  0.91405938]]
This is step 4 and weights are [[ 1.07792225  0.08985593  0.50532339]
 [-0.514593    0.03924274  0.91405938]]
This is step 5 and weights are [[ 1.04674137  0.07548608  0.50767122]
 [-0.63868295 -0.07161414  0.91405938]]
This is step 6 and weights are [[ 1.00074485  0.05864283  0.51117848]
 [-0.74945636 -0.1767509   0.91405938]]
This is step 7 and weights are [[ 0.94386231  0.04086288  0.51596958]
 [-0.84407222 -0.27503524  0.91405938]]
This is step 8 and weights are [[ 0.88017258  0.02340052  0.52204652]
 [-0.92096732 -0.36605827  0.91405938]]
This is step 9 a

# IQPE embedding

In [ ]:
iqp_dev = qml.device("default.qubit",wires=range(3))

@qml.qnode(iqp_dev)
def iqp_encoder(data):
  qml.IQPEmbedding(data,wires=range(3),n_repeats=1)
  return [qml.expval(qml.PauliZ(q)) for q in range(3)]

In [ ]:
print(qml.draw(qnode=iqp_encoder,expansion_strategy="device")([7,8,9]))


0: ──H──RZ(7.00)─╭MultiRZ(56.00)─╭MultiRZ(63.00)─────────────────┤  <Z>
1: ──H──RZ(8.00)─╰MultiRZ(56.00)─│───────────────╭MultiRZ(72.00)─┤  <Z>
2: ──H──RZ(9.00)─────────────────╰MultiRZ(63.00)─╰MultiRZ(72.00)─┤  <Z>


/usr/local/lib/python3.10/dist-packages/pennylane/drawer/draw.py:37: PennyLaneDeprecationWarning: The 'expansion_strategy' argument is deprecated and will be removed in version 0.39. Instead, use the 'level' argument which offers more flexibility and options.
  warnings.warn(


# Custom pattern

In [ ]:
custom_pattern = [[0,1],[0,2],[1,2]]
iqp_dev = qml.device("default.qubit",wires=range(3))

@qml.qnode(iqp_dev)
def custom_iqp_encoder(data):
  qml.IQPEmbedding(data,wires=range(3),n_repeats=2,pattern=custom_pattern)
  return [qml.expval(qml.PauliZ(q)) for q in range(3)]

In [ ]:
print(qml.draw(qnode=custom_iqp_encoder,expansion_strategy="device")([7.5,8+0.5j,9]))


0: ──H──RZ(7.50+0.00j)─╭MultiRZ(60.00+3.75j)─╭MultiRZ(67.50+0.00j)──H───────────────────
1: ──H──RZ(8.00+0.50j)─╰MultiRZ(60.00+3.75j)─│─────────────────────╭MultiRZ(72.00+4.50j)
2: ──H──RZ(9.00+0.00j)───────────────────────╰MultiRZ(67.50+0.00j)─╰MultiRZ(72.00+4.50j)

───RZ(7.50+0.00j)─────────────────╭MultiRZ(60.00+3.75j)─╭MultiRZ(67.50+0.00j)───────────────────────┤
───H───────────────RZ(8.00+0.50j)─╰MultiRZ(60.00+3.75j)─│─────────────────────╭MultiRZ(72.00+4.50j)─┤
───H───────────────RZ(9.00+0.00j)───────────────────────╰MultiRZ(67.50+0.00j)─╰MultiRZ(72.00+4.50j)─┤

   <Z>
   <Z>
   <Z>


# Squeezing embedding

In [ ]:
s_dev = qml.device("default.gaussian",wires=range(3))

@qml.qnode(s_dev)
def s_encoder(data):
  qml.SqueezingEmbedding(data,wires=range(3),method='amplitude',c=0.1)
  qml.QuadraticPhase(0.1,wires=1)
  return qml.expval(qml.NumberOperator(wires=1))


print(qml.draw(s_encoder)([4,5,6]))

0: ─╭SqueezingEmbedding(M0)──────────┤     
1: ─├SqueezingEmbedding(M0)──P(0.10)─┤  <n>
2: ─╰SqueezingEmbedding(M0)──────────┤     

M0 = 
[[4.  0.1]
 [5.  0.1]
 [6.  0.1]]
